In [1]:
import pandas as pd

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# PATH = '/content/drive/MyDrive/Projects/RBK/'

In [2]:
train = pd.read_csv('train_dataset_train.csv')
test = pd.read_csv('test_dataset_test.csv')

In [3]:
df = pd.concat([train, test], ignore_index=True)

In [4]:
df['publish_date'] = pd.to_datetime(df['publish_date'])

In [5]:
df['year']=df.publish_date.dt.year

In [6]:
df['month']=df.publish_date.dt.month
df['day']=df.publish_date.dt.day

In [10]:
df.columns

Index(['document_id', 'title', 'publish_date', 'session', 'authors', 'ctr',
       'category', 'tags', 'views', 'depth', 'full_reads_percent', 'year',
       'month', 'day'],
      dtype='object')

In [7]:
df = df.drop(columns=['session', 'authors', 'ctr',
      'tags', 'views', 'depth', 'full_reads_percent'], axis=1)

In [8]:
df['id']=df.document_id.apply(lambda x: x[0:24])

In [9]:
def dt_con(x):
    a=0
    if x < 10:
        a=str(0)+str(x)
    else:
        a=str(x)
    return a

In [10]:
df.month = df.month.apply(lambda x: dt_con(x))
dfday_new=df.day.apply(lambda x: dt_con(x))

In [11]:
def cat(x):
    a=0
    if x == '5409f11ce063da9c8b588a12':
        a = 'politics'
    elif x == '5433e5decbb20f277b20eca9':
        a = 'society'
    elif x == '5409f11ce063da9c8b588a13':
        a = 'economics'
    elif x == '540d5eafcbb20f2524fc0509' or x == '540d5ecacbb20f2524fc050a':
        a = 'rbcfreenews' # no data
    elif x == '5409f11ce063da9c8b588a18':
        a = 'finances'
    elif x == '5e54e2089a7947f63a801742':
        a = 'auto'
    elif x == '552e430f9a79475dd957f8b3':
        a = 'money'
    elif x == '5e54e22a9a7947f560081ea2':
        a = 'realty'
    return a


In [12]:
df.category =df.category.apply(lambda x: cat(x))

In [13]:
df['url'] = 'https://www.rbc.ru'
# url = 'https://www.rbc.ru'

In [14]:
df.year = df.year.astype('str')
df.day = df.day.astype('str')

In [ ]:
# df['year_new']=df['year'].apply(lambda x: dt_con(x))

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   document_id   10000 non-null  object        
 1   title         10000 non-null  object        
 2   publish_date  10000 non-null  datetime64[ns]
 3   category      10000 non-null  object        
 4   year          10000 non-null  object        
 5   month         10000 non-null  object        
 6   day           10000 non-null  object        
 7   id            10000 non-null  object        
 8   url           10000 non-null  object        
dtypes: datetime64[ns](1), object(8)
memory usage: 703.2+ KB


In [16]:
df['url2']= df[['url','category', 'day', 'month', 'year','id']].agg('/'.join, axis=1)

In [17]:
import requests
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [18]:
def sub(url):
    b = []
    response = requests.get(url)
    dom = bs(response.text, 'html.parser')
    if response.ok:

        sub = dom.find_all('div', class_="article__text__overview")
        b.append(sub)

    return b


In [19]:
def bod(url):
    b = []
    response = requests.get(url)
    dom = bs(response.text, 'html.parser')
    if response.ok:
        body = dom.find_all('p')
        b.append(body)

    return b


In [32]:
b=df['url2'][10]

In [40]:
df.head(2)

,document_id,title,publish_date,category,year,month,day,id,url,url2,prs
0,624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ ...,2022-04-04 10:29:44,finances,2022,04,4,624ac09c9a7947db3d80c98e,https://www.rbc.ru,https://www.rbc.ru/finances/4/04/2022/624ac09c...,0
1,620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные зап...,2022-02-18 10:00:39,politics,2022,02,18,620f6b899a7947701cf489e1,https://www.rbc.ru,https://www.rbc.ru/politics/18/02/2022/620f6b8...,0


In [21]:
df['subtitle']='0'
df['body']='0'

In [22]:
for i in range(0, 2000):
    df['subtitle'][i]=sub(df['url2'][i])
    df['body'][i]=bod(df['url2'][i])

/var/folders/v8/9ljqp91d0pz2t93nf7r4kqjh0000gn/T/ipykernel_14015/3635217114.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subtitle'][i]=sub(df['url2'][i])
/var/folders/v8/9ljqp91d0pz2t93nf7r4kqjh0000gn/T/ipykernel_14015/3635217114.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['body'][i]=bod(df['url2'][i])


In [24]:
for i in range(2000, 10000):
    df['subtitle'][i]=sub(df['url2'][i])
    df['body'][i]=bod(df['url2'][i])

/var/folders/v8/9ljqp91d0pz2t93nf7r4kqjh0000gn/T/ipykernel_14015/960364172.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subtitle'][i]=sub(df['url2'][i])
/var/folders/v8/9ljqp91d0pz2t93nf7r4kqjh0000gn/T/ipykernel_14015/960364172.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['body'][i]=bod(df['url2'][i])


In [29]:
df[['title','subtitle','body']].head(20)

,title,subtitle,body
0,Европейский банк развития приостановил доступ ...,[],[]
1,Кремль назвал регулярным процессом учебные зап...,"[[[\n, <span>Все страны, которым положено знат...",[[[Учения стратегических сил с запусками балли...
2,Госсекретарь Швеции заявила о нежелании вступа...,[[]],[[[Эскалация на Украине не дала повода Швеции ...
3,Песков назвал прагматичной выдачу лицензии Газ...,[[]],[[[Решение Великобритании выдать Газпромбанку ...
4,В Хабаровске задержали главу филиала РАНХиГС п...,[[]],[[[Глава дальневосточного филиала Российской а...
5,Глава ВОЗ заявил о росте смертности из-за COVI...,[],[]
6,Третий за день губернатор объявил об уходе,"[[[\n, <span>Вслед за главами Томской и Кировс...",[[[Губернатор Рязанской области Николай Любимо...
7,Губернатор сообщил о попытке самолета ВСУ попа...,"[[[\n, <span>По словам Александра Богомаза, дв...",[[[Российская система ПВО не допустила проникн...
8,Антонов заявил о межпартийном консенсусе США п...,[],[]
9,Опубликовавшая время вторжения The Mirror напи...,[[]],[[[Опубликовавшая время вторжения России на Ук...


In [30]:
df=df.drop(columns=['year', 'day', 'month', 'url', 'url2', 'publish_date', 'id'], axis=1)

In [31]:
df.head()

,document_id,title,category,subtitle,body
0,624ac09c9a7947db3d80c98eIDE7mtH4RBqGn-8MXfGffQ,Европейский банк развития приостановил доступ ...,finances,[],[]
1,620f6b899a7947701cf489e1KtVJsteHStO5oditt3Uvzw,Кремль назвал регулярным процессом учебные зап...,politics,"[[[\n, <span>Все страны, которым положено знат...",[[[Учения стратегических сил с запусками балли...
2,620730cf9a7947ab96a44e27hk7puWJwSziw0m3sfTkKWA,Госсекретарь Швеции заявила о нежелании вступа...,politics,[[]],[[[Эскалация на Украине не дала повода Швеции ...
3,6262a5889a79470b78c9ca307UKY2SSZTjCcjhwBzxw37w,Песков назвал прагматичной выдачу лицензии Газ...,politics,[[]],[[[Решение Великобритании выдать Газпромбанку ...
4,626678929a79477ca0101568wuMYES90REuV5YhrN75IXg,В Хабаровске задержали главу филиала РАНХиГС п...,society,[[]],[[[Глава дальневосточного филиала Российской а...


In [32]:
df.to_csv('text_dirty.csv')